In [1]:
import time     #for testing 
import os       #for testing  

import requests
import bs4
from bs4 import BeautifulSoup, Comment  
import pandas as pd
import numpy as np
##Selenium won't be used in the final product
from selenium import webdriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager

In [2]:
#wont be using driver in the end, found a more effective method
#driver = webdriver.Edge('.\msedgedriver.exe')

<h4>Functions:</h4>
    <div>*load_soup_obj - receives a url and returns a soup object.</div></n>
    <BR>	
    <div>*get_page_values - receives a soup object of an anime page, gets the types (columns for the dataframe) and the values           of said types, returns them as a dictionary with keys and values.</div></n>
    <BR>	
    
   <div>*create_data_frame - receives a list of urls, each url leads to an anime page, gets each anime types and values with            get_page_values (because the run time for this function is so long O(N), I placed a percentage printer for the 10k              samples of the first data base, and a time print in the end to see how long it takes(averages in around 2hours for 10k          samples)), puts them in a DataFrame and returns it.</div>

In [3]:
def load_soup_obj(url):
    path = requests.get(url)
    soup_path = path.text
    soup = BeautifulSoup(soup_path, "html.parser")
    return soup

In [4]:
def get_page_values(soup):
    type_list = soup.find_all("div", attrs ={"class":"type"})
    temp_value_list = soup.find_all("div", attrs ={"class":"value"})
    key_list = []
    value_list=[]
    for key in type_list:
        insert_key = key.get_text(strip=True)
        if(insert_key == 'Season'):
            continue
        key_list.append(insert_key)
    for val in temp_value_list:
        insert_value = val.get_text(strip=True)
        value_list.append(insert_value)
    dic = dict(zip(key_list, value_list))
    return dic

In [5]:
def scroll_screen(driver):
    scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1
    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break 


In [6]:
def create_data_frame(broken_url_list):
    df = pd.DataFrame()
    start_time = time.time()
    index = 0
    for url_list in broken_url_list:
        for url in url_list:
            index = index + 1
            if(index %500 == 0):
                print(f'%', index/100,' Complete')
            new_url = url.get_text()
            temp_soup = load_soup_obj(new_url)
            url_dic = get_page_values(temp_soup)
            df = df.append(url_dic, ignore_index=True)
    end_time = time.time()
    print(f"it takes:", end_time-start_time, "to make a dataframe of 10k examples")       
    return df

<h4>Main body:</h4>
    <div>get the url list from the sitemap .xml page, and creates a dataframe using the create_data_frame function, then saves it to     a .csv format in order to avoid running the crawler everytime we need the data. 
    (formerly I used selenium to scroll on the page to get each url, but the load time, scroll time and getting the url were       really inefficient, thus I searched for a more efficient method and found the sitemap)</div>

In [8]:
broken_url_list = []
main_soup = load_soup_obj('https://anilist.co/sitemap/anime-1.xml')
broken_url_list.append(main_soup.find_all("loc"))

In [ ]:
df = create_data_frame(broken_url_list)
df

In [ ]:
df.to_csv('Anime_Data_Frame_2')